# РЕКОМЕНДАТЕЛЬНЫЕ СИСТЕМЫ ЧАСТЬ 2

### **Content-based model**

In [2]:
import pandas as pd

In [1]:
import numpy as np

v = np.array([1.1,2.3,5.1])
w = np.array([5.1,6.2,1.1])

v@w / (np.linalg.norm(v)*np.linalg.norm(w))

0.5514936289973482

* Для каждого продукта создать характеризующие его признаки.
* Найти показатель близости между всеми продуктами.
* Порекомендовать пользователю продукты, которые показывают наибольшую близость с теми продуктами, которые он высоко оценил.

In [3]:
df = pd.read_csv('netflix_titles.csv')
df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...


В первую очередь нам необходимо определить, на основании чего мы будем рассматривать близость фильмов. Выберем для этой задачи описание фильма, ведь в нём, скорее всего, содержится много информации. Однако описание — это текст. Есть много подходов к преобразованию текста в вектор, и мы будем использовать подход TF-IDF (Term Frequency-Inverse Document Frequency).

Показатель TD-IDF — это индикатор того, насколько релевантно слово в контексте документа.

$\text{TF-IDF(слова) = TF(слова) * IDF (слова)}$

$\text{TF слова} = \frac{\text{Количество раз, когда слово встретилось в тексте}}{\text{Количество всех слов в тексте}}$

$\text{IDF слова} = log \left (\frac{\text{Общее кол-во документов}}{\text{Кол-во документов, в которых встречается слово}}\right )$

Этот показатель возрастает пропорционально количеству раз, когда слово встречается в тексте, и уменьшается пропорционально количеству слов во всех текстах в целом.

Таким образом:

*Коэффициент будет выше*, если слово характерно именно для этого текста, то есть встречается в данном тексте часто, но не встречается в других текстах.

*Коэффициент будет ниже*, если слово не встречается почти нигде или встречается одинаковое количество раз во всех текстах, то есть не характеризует никакой текст в отдельности.

In [4]:
#Чтобы преобразовать текст по этому принципу, нам понадобится соответствующая функция из библиотеки sklearn — импортируем её:

from sklearn.feature_extraction.text import TfidfVectorizer
#Далее учтём стоп-слова, т. е. предлоги и другие служебные части речи, 
# которые не несут содержательной информации, и с учётом этого определим нашу модель:

model = TfidfVectorizer(stop_words='english')
#Заполним пропуски пустыми строками:

df['description'] = df['description'].fillna('')
#Трансформируем наши описания в матрицу:

feature_matrix = model.fit_transform(df['description'])

In [6]:
feature_matrix.shape

(7787, 17905)

In [8]:
#Теперь необходимо вычислить косинусную близость. Можно сделать это так:

from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

Обратите внимание! Мы используем здесь linear_kernel(), а не cosine_similarity(), так как в косинусном расстоянии в знаменателе реализуется нормировка векторов, а TF-IDF создаёт уже нормализованные векторы.

In [12]:
#Вернём индексацию и уберём дубликаты из данных:

indices = pd.Series(df.index,index=df['title']).drop_duplicates()

In [15]:
#Теперь пропишем функцию для создания рекомендаций:

def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

In [17]:
get_recommendations('Balto')

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

*плюсы*

* Для создания рекомендаций не требуются данные от других пользователей. Как только пользователь выполнил поиск, просмотрел несколько продуктов и/или совершил несколько покупок, система фильтрации на основе контента может начать создавать соответствующие рекомендации. Это делает её идеальной для компаний и сервисов, у которых нет огромного количества пользователей для формирования выборки.

* Рекомендации получаются очень релевантными для пользователя. Рекомендации на основе контента могут быть в значительной степени адаптированы к интересам пользователя, включая рекомендации по нишевым товарам, поскольку метод основан на сопоставлении характеристик или атрибутов объекта базы данных с интересами пользователя.

* Рекомендации прозрачны для пользователя. Высокорелевантные рекомендации создают ощущение понятности алгоритмов для пользователя, повышая уровень его доверия к предлагаемым рекомендациям.

* Вы избегаете проблемы «холодного старта». Хотя фильтрация на основе контента требует первоначального ввода данных от пользователей, чтобы начать давать рекомендации, качество ранних рекомендаций обычно намного выше, чем у других подходов.

* Системы фильтрации на основе содержания обычно проще в создании. Основная работа заключается в создании характеристик, на основании которых будет вычисляться близость.

*минусы*

* Отсутствие новизны и разнообразия. Вполне возможно, что человеку, который любит боевики, могли бы понравиться и фильмы ужасов. Однако мы не сможем ему их порекомендовать, если он сам не отметит подобные фильмы как понравившиеся.

* Присвоенные характеристики могут быть неверными. Рекомендации на основе контента хороши настолько, насколько хороши атрибуты (характеристики), присвоенные продуктам.

### **Коллаборативная фильтрация**

 Следующий подход к построению рекомендательных систем, который мы изучим, — это коллаборативная фильтрация. Она основана на поиске сходства между пользователями или между продуктами. Используя этот метод, мы можем прогнозировать рейтинги на основе оценок похожих пользователей или похожих продуктов.

Однако для начала давайте рассмотрим очень часто встречающуюся в рекомендательных системах концепцию — матрицу предпочтений.

Чтобы её получить, расположим в матрице клиентов по строкам, а продукты — по столбцам. На пересечении строк и столбцов разместим оценки, поставленные клиентами соответствующим продуктам: первый клиент поставил второму товару 3, третий клиент поставил первому товару 2 и так далее.

На основе этих данных мы можем разделить пользователей на кластеры. Чтобы это сделать, можно взять некоторую меру близости для пользователей по их истории оценок и на основе полученных значений объединить пользователей в кластеры таким образом, чтобы похожие пользователи оказались в одной группе, а сильно отличающиеся — в разных. В таком случае оценку пользователя для продукта можно прогнозировать как среднюю оценку пользователей этого кластера, оценивших этот продукт.

Таким образом, если нам необходимо предсказать, как конкретный пользователь оценил фильм, мы анализируем оценки, поставленные данному фильму пользователями, которые принадлежат к тому же кластеру, что и изучаемый пользователь, и усредняем эти оценки. Так получается предсказание оценки фильма для нашего пользователя.

В целом, такой подход можно применять, однако у него есть ряд существенных недостатков:

* Нечего рекомендовать новым/нетипичным пользователям. Если появляется пользователь, который ни на кого не похож, мы не знаем, к какому кластеру его отнести. На начальных стадиях мы определяем его в случайный кластер, и рекомендации в таком случае будут плохими.
* Не учитывается специфика каждого пользователя. По сути, мы выявляем некоторые паттерны поведения и предпочтений и для каждого паттерна выделяем свои рекомендации. Однако на самом деле даже пользователи из одного кластера немного отличаются друг от друга, поэтому возникают неточности.
* Если оценок нет, то среднее арифметическое невозможно вычислить. Если в кластере никто не оценивал объект, сделать предсказание не получится, так как для предсказания нужно вычислить среднее арифметическое для оценок.

**КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ НА ОСНОВЕ ПАМЯТИ (MEMORY-BASED)**

Чтобы решить перечисленные выше проблемы, обратимся к коллаборативной фильтрации, а точнее к memory-based-подходу, основанному на близости пользователей (user-based).

Напомним, что при memory-based-подходе хранится полная матрица взаимодействий (лайков, просмотров и т. д .) пользователя с продуктом.

Коллаборативная фильтрация на основе пользователей — это метод, используемый для предсказания продуктов, которые могут понравиться пользователю, на основе оценок, выставленных этому продукту другими пользователями, имеющими схожие с целевым пользователем вкусы.